In [2]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

In [3]:
pwd

'/opt/ml/code/mmdetection_trash'

In [3]:
# !wget http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth

In [4]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')
cfg.work_dir = './work_dirs/sudo_faster_rcnn_r50_fpn_1x_trash'
cfg.load_from = 'faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'

PREFIX = '../../input/data/'

cfg.data.samples_per_gpu = 8
cfg.data.workers_per_gpu = 4

# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = [PREFIX, PREFIX]
cfg.data.train.ann_file = [PREFIX + 'train.json', PREFIX + '0.9_0.9_test_psuedo.json']
cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

cfg.seed=2020
cfg.gpu_ids = [0]
cfg.runner.max_epochs = 15

cfg.model.roi_head.bbox_head.num_classes = 11

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    type='FasterRCNN',
    pretrained='torchvision://resnet50',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch'),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0),
        loss_bbox=dict(typ

In [5]:
import pickle
with open('sodu_config_standard', 'wb') as f:
    pickle.dump(cfg, f)

In [6]:
model = build_detector(cfg.model)

2021-05-18 09:18:12,195 - mmdet - INFO - load model from: torchvision://resnet50
2021-05-18 09:18:12,196 - mmdet - INFO - Use load_from_torchvision loader
2021-05-18 09:18:12,450 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias



In [7]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=3.78s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [8]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

loading annotations into memory...


2021-05-18 09:18:22,591 - mmdet - INFO - load checkpoint from faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
2021-05-18 09:18:22,592 - mmdet - INFO - Use load_from_local loader


Done (t=0.88s)
creating index...
index created!


2021-05-18 09:18:23,394 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([12, 1024]).
size mismatch for roi_head.bbox_head.fc_cls.bias: copying a param with shape torch.Size([81]) from checkpoint, the shape in current model is torch.Size([12]).
size mismatch for roi_head.bbox_head.fc_reg.weight: copying a param with shape torch.Size([320, 1024]) from checkpoint, the shape in current model is torch.Size([44, 1024]).
size mismatch for roi_head.bbox_head.fc_reg.bias: copying a param with shape torch.Size([320]) from checkpoint, the shape in current model is torch.Size([44]).
2021-05-18 09:18:23,399 - mmdet - INFO - Start running, host: root@55fddde1a768, work_dir: /opt/ml/code/mmdetection_trash/work_dirs/sudo_faster_rcnn_r50_fpn_1x_trash
2021-05-18 09:18:23,400 - mmdet - INFO - workflow: [('train', 1)], max

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 655/655, 22.1 task/s, elapsed: 30s, ETA:     0s

...

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 655/655, 22.3 task/s, elapsed: 29s, ETA:     0s

2021-05-18 10:18:06,322 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.09s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=5.07s).
Accumulating evaluation results...


2021-05-18 10:18:12,226 - mmdet - INFO - Epoch(val) [15][353]	bbox_mAP: 0.2310, bbox_mAP_50: 0.3810, bbox_mAP_75: 0.2310, bbox_mAP_s: 0.0450, bbox_mAP_m: 0.2040, bbox_mAP_l: 0.4300, bbox_mAP_copypaste: 0.231 0.381 0.231 0.045 0.204 0.430


DONE (t=0.69s).
Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.231
Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.381
Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.231
Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.045
Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.204
Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.430
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.361
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.361
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.361
Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.113
Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.330
Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.618
